# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

--- 

# Importing Libraries and Loading Data

Importing the necessary Python libraries and loads data from JSON files into Pandas DataFrames.

### Libraries Imported
- `pandas` (`pd`): Used for data manipulation and analysis.
- `numpy` (`np`): Provides support for numerical operations.
- `math`: Standard Python library for mathematical functions.
- `json`: Enables working with JSON data.
- `matplotlib.pyplot` (`plt`): Used for data visualization.
- `seaborn` (`sns`): Enhances visualization with statistical plotting.

### Loading Data
The `pd.read_json()` function is used to read JSON files:
- `portfolio_raw`: Contains data from `portfolio.json`, likely representing promotional offers.
- `profile_raw`: Contains data from `profile.json`, likely storing user demographic information.
- `transcript_raw`: Contains data from `transcript.json`, likely recording user interactions or transactions.

Each file is read with `orient='records'` and `lines=True`, ensuring that each JSON object in the file is interpreted as a separate record (suitable for line-delimited JSON files).


In [60]:
# importing libraries
import pandas as pd
import numpy as np

# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)


# Data understanding

--- 
Objectives: 

* Examination of each individual table and its corresponding columns.
* Exploratory data analysis (EDA) with some statistics.
---


**portfolio.json:**

 Ten offers type and his atributes.

-> Data is cleaned and ready to be used

In [61]:
portfolio_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [62]:
# showing the entire table
portfolio_raw

,reward,channels,difficulty,duration,offer_type,id
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5


**profile.json**

Customers demographic data.

--> There are 17000 customers in the dataset.

--> There are 2175 (~12,7 %) NoneType values in `gender` and `income`, and the `age` values is 118 for these.

--> ~50 % are 'M' and ~36 % are 'F'. There is ~1.2 %  'O' type gender.

--> The colum `became_member_on` has non-formated date values.

In [63]:
profile_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 664.2+ KB


In [64]:
profile_raw['gender'].value_counts(dropna=False, normalize=True)

gender
M       0.499059
F       0.360529
None    0.127941
O       0.012471
Name: proportion, dtype: float64

In [65]:
profile_raw['age'].value_counts(dropna=False, normalize=True)

age
118    0.127941
58     0.024000
53     0.021882
51     0.021353
59     0.021118
         ...   
100    0.000706
96     0.000471
98     0.000294
101    0.000294
99     0.000294
Name: proportion, Length: 85, dtype: float64

In [66]:
profile_raw.describe(include='all') 

,gender,age,id,became_member_on,income
count,14825,17000.000000,17000,1.700000e+04,14825.000000
unique,3,NaN,17000,NaN,NaN
top,M,NaN,e4052622e5ba45a8b96b59aba68cf068,NaN,NaN
freq,8484,NaN,1,NaN,NaN
mean,NaN,62.531412,NaN,2.016703e+07,65404.991568
std,NaN,26.738580,NaN,1.167750e+04,21598.299410
min,NaN,18.000000,NaN,2.013073e+07,30000.000000
25%,NaN,45.000000,NaN,2.016053e+07,49000.000000
50%,NaN,58.000000,NaN,2.017080e+07,64000.000000
75%,NaN,73.000000,NaN,2.017123e+07,80000.000000


**transcript.json**

A time line of the events that took place during the simulation event.

--> There is a inconsistent dicttionary keys in the `value` column.

--> The `' '` in the `event` column categorie's names can be normalized to `'_'`.

--> There are no missing values.

In [67]:
transcript_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 9.4+ MB


In [68]:
{[*x][0] for x in transcript_raw['value']}

{'amount', 'offer id', 'offer_id'}

In [69]:
transcript_raw['event'].value_counts()

event
transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: count, dtype: int64

# Data Preparation
---
- Objective:

Creation of `Analytical Tables` datasets for analisys, visuals recommendations and machine learning applications.

- Strategy:
1. Loading raw data from the original tables.
2. Restructuring it using groupby/unstack and creating fetures.
3. Selecting relevant variables and fetures.

This process ensures that the data tables and **features** is properly formatted, aggregated, and cleaned for further analysis. 

---

#### **Portfolio Dataset**
- Update column `id` to make them more easier to read.

In [70]:
#creating a copy from the original dataframe
portfolio = portfolio_raw.copy() 

# renaming the columns using a dictionary
port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

# mapping the id column
portfolio['ofr_id_short'] = portfolio['id'].map(port_id)

# persist a csv file to the bronze folder
portfolio.to_csv('medalion_data_store/bronze/portfolio.csv', index=False)

portfolio

,reward,channels,difficulty,duration,offer_type,id,ofr_id_short
0,10,"[email, mobile, social]",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd,ofr_A
1,10,"[web, email, mobile, social]",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,ofr_B
2,0,"[web, email, mobile]",0,4,informational,3f207df678b143eea3cee63160fa8bed,ofr_C
3,5,"[web, email, mobile]",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,ofr_D
4,5,"[web, email]",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,ofr_E
5,3,"[web, email, mobile, social]",7,7,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,ofr_F
6,2,"[web, email, mobile, social]",10,10,discount,fafdcd668e3743c1bb461111dcafc2a4,ofr_G
7,0,"[email, mobile, social]",0,3,informational,5a8bc65990b245e5a138643cd4eb9837,ofr_H
8,5,"[web, email, mobile, social]",5,5,bogo,f19421c1d4aa40978ebb69ca19b0e20d,ofr_I
9,2,"[web, email, mobile]",10,7,discount,2906b810c7d4411798c6938adc9daaa5,ofr_J


#### **Profile Dataset**
- Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.
-  Create a new column with only the year and month of the date.

In [71]:
def categorize_age(age):
    """
    Categorizes the input age into a predefined group.

    Parameters:
    age (int): The age to categorize.

    Returns:
    str: The category the age falls into. One of 'Young', 'Adult', 
         'Middle', or 'Senior'.
    """
    if age < 25:
        return 'Young'
    elif 25 <= age < 40:
        return 'Adult'
    elif 40 <= age < 60:
        return 'Middle'
    else:
        return 'Senior'


In [72]:
# copy the raw data into a new dataframe
profile = profile_raw.copy(deep=True)

# Convert the 'became_member_on' column to a datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# Create a new column with only the year and month of the membership
profile['bec_memb_year_month'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d').dt.strftime('%Y-%m')

# appling the age categoization function    
profile['age_group'] = profile['age'].apply(categorize_age).astype('category')

# Persist a csv file to the data store
profile.to_csv('medalion_data_store/bronze/profile.csv')

profile.head()

,gender,age,id,became_member_on,income,bec_memb_year_month,age_group
0,None,118,68be06ca386d4c31939f3a4f0e3dd783,2017-02-12,NaN,2017-02,Senior
1,F,55,0610b486422d4921ae7d2bf64640c50b,2017-07-15,112000.0,2017-07,Middle
2,None,118,38fe809add3b4fcf9315a9694bb96ff5,2018-07-12,NaN,2018-07,Senior
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,2017-05,Senior
4,None,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,NaN,2017-08,Senior


#### **Transcript dataset**
- Clean, normalizing and transforming column values from the transcript orighinal table.
- Creating a new table `transcript_b` using json_normalize() and concat() methods.

**Strategy:**

1. Copy the data.
2. Normalizing the `value` column dictionarie keys `offer id` --> `offer_id`.
3. Creating the `transcript_b` table using `pd.json_normalize()` and `pd.concat()` functions.
3. Normalizing the `event` column values replacing `' '` --> `'_'`. 
4. Creating `ofr_id_short` column with an id more readeble and droping the `offer_id` column.
5. Fill NaN with apropriate values.
6. Creating a `tag` column to identify the person-event-offer interactions one by one. (as a person can interact with the same offer type more than once)
7. Persist the table in a csv file and save in the data store.

In [73]:
def fix_offer_id(value):
    """
    Fixes the 'offer id' key in a dictionary by renaming it to 'offer_id'.

    Parameters:
    value (dict): A dictionary that may contain the 'offer id' key.

    Returns:
    dict: The updated dictionary with 'offer id' replaced by 'offer_id'.
    """
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

In [74]:
# copy the raw data into a new dataframe
transcript = transcript_raw.copy(deep=True)


# appling the fix offer function
transcript['value'] = transcript['value'].apply(fix_offer_id)

# Normalize the 'value' column with json_normalize method
value_df = pd.json_normalize(transcript['value']) 
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

# Normalizing the event column categorie's names
transcript_b['event'] = [x.split(' ')[1] if len(x.split(' ')) > 1 else x for x in transcript_b['event']  ]

# mapping the offer_id to the offer_id_short and Dropping the offer_id column
transcript_b['ofr_id_short'] = transcript_b['offer_id'].map(port_id).fillna('tran')
transcript_b = transcript_b.drop(columns = ['offer_id'])

# creating a tag column to identify the order of the events for each person-offer-event fact
transcript_b['tag'] = (
    transcript_b.groupby(['person', 'ofr_id_short', 'event'], observed=True)
    .cumcount()
)

# persist a csv file to the bronze folder
transcript_b.to_csv('medalion_data_store/bronze/transcript_b.csv', index=False)

transcript_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   person        306534 non-null  object 
 1   event         306534 non-null  object 
 2   time          306534 non-null  int64  
 3   amount        138953 non-null  float64
 4   reward        33579 non-null   float64
 5   ofr_id_short  306534 non-null  object 
 6   tag           306534 non-null  int64  
dtypes: float64(2), int64(2), object(3)
memory usage: 16.4+ MB


### Separating *transactions* events in transcript_b dataset from the others

* **Source**: 

    `transcript_b`.

* **Created tables:**

    `events` and `transactions`: separated data from `transcript_b`


- creating `events` and `transactions` dataframes
- Dropping columns that are not needed

In [75]:
# filtering  the data that are different than transaction
events = transcript_b[transcript_b['event'] != 'transaction'].copy()

# drop the 'amount' column as it contais zero for all rows.
events = events.drop(columns=['amount']) 

# filtering  the transaction data 
transactions = transcript_b[transcript_b['event'] == 'transaction'].copy()

# drop the reward (all zeros) and offer_id columns as they are not relevant in this dataset. 
transactions = transactions.drop(columns=['reward',	'ofr_id_short']) 

# persisting the data in the silver layer
events.to_csv('medalion_data_store/bronze/events.csv', index=False)
transactions.to_csv('medalion_data_store/bronze/transactions.csv', index=False)

In [76]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167581 entries, 0 to 306527
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   person        167581 non-null  object 
 1   event         167581 non-null  object 
 2   time          167581 non-null  int64  
 3   reward        33579 non-null   float64
 4   ofr_id_short  167581 non-null  object 
 5   tag           167581 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 8.9+ MB


In [77]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138953 entries, 12654 to 306533
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   person  138953 non-null  object 
 1   event   138953 non-null  object 
 2   time    138953 non-null  int64  
 3   amount  138953 non-null  float64
 4   tag     138953 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 6.4+ MB


# **Features Engineering**

- Process the datasets to create useful tables and extract fetures from the data.

## Ranking the Offers

Using events table to ranking the offers by counts of the (`event`) column categories and create rate features

In [78]:
# Grouping the events DataFrame by 'ofr_id_short' and 'event' to count the occurrences
offer_rank = events.groupby(['ofr_id_short', 'event']).agg(
    cnt=('event', 'count'),
).unstack(level=[1]).reset_index().fillna(0).round(2)

# Flattening multi-level column names and joining them with an underscore
offer_rank.columns = ['_'.join(col).strip('_') for col in offer_rank.columns.to_flat_index()]

# Calculating the completion and viewed rates based on event counts
offer_rank['completion_rate'] = offer_rank['cnt_completed'] / offer_rank['cnt_received']
offer_rank['viewed_rate'] = offer_rank['cnt_viewed'] / offer_rank['cnt_received']

offer_rank =offer_rank.round(2)
# Saving the resulting DataFrame to a CSV file
offer_rank.to_csv('medalion_data_store/silver/offer_rank.csv', index=False)

# Returning the offer_rank DataFrame
offer_rank


,ofr_id_short,cnt_completed,cnt_received,cnt_viewed,completion_rate,viewed_rate
0,ofr_A,3688.0,7658.0,6716.0,0.48,0.88
1,ofr_B,3331.0,7593.0,7298.0,0.44,0.96
2,ofr_C,0.0,7617.0,4144.0,0.00,0.54
3,ofr_D,4354.0,7677.0,4171.0,0.57,0.54
4,ofr_E,3420.0,7668.0,2663.0,0.45,0.35
5,ofr_F,5156.0,7646.0,7337.0,0.67,0.96
6,ofr_G,5317.0,7597.0,7327.0,0.70,0.96
7,ofr_H,0.0,7618.0,6687.0,0.00,0.88
8,ofr_I,4296.0,7571.0,7264.0,0.57,0.96
9,ofr_J,4017.0,7632.0,4118.0,0.53,0.54


## User-Item-Event matrix:

### A table that tracks interactions based on **person** and **ofr_id_short** columns. The aggregation process computes several statistics related to the **event** count, **reward**, and **time** for each user-item interaction.
---

**Source table**: 

  `events` table: providing event-type, time-stamp and reward per person interactions with offers.

**Created table:**
  
  `user_item_events`: grouped table by `'person'` and `'ofr_id_short'` with counts and statistics features.

**Columns Created:**

  1. **cnt_eve**: Total number of events per (person, ofr_id_short) pair.
  2. **sum_rew_eve**: Sum of rewards.
  3. **mean_rew_eve**: Mean of rewards.
  4. **median_rew_eve**: Median of rewards.
  5. **std_rew_eve**: Standard deviation of rewards. **(*removed*)**
  6. **min_rew_eve**: Minimum reward value.
  7. **max_rew_eve**: Maximum reward value.
  8. **range_rew_eve**: Range of rewards (max - min).
  9. **mean_t_eve**: Mean event time.
  10. **median_t_eve**: Median event time.
  11. **std_t_eve**: Standard deviation of event times. **(*removed*)**
  12. **min_t_eve**: Minimum event time.
  13. **max_t_eve**: Maximum event time.
  14. **range_t_eve**: Range of event times (max - min).
  15. **freq_eve**: Frequency of events per (person, ofr_id_short). If max time equals min time, frequency is set to 0 to avoid division by zero.
  16. **last_to_end_eve**: last event occurrence before the end of offer program (end=714h).

**Handling Missing Values:**
  - fill NaN with zero.
---

In [79]:
user_item_events = events.groupby(['person', 'ofr_id_short']).agg(
    cnt_eve=('event', 'count'),        
    sum_rew_eve=('reward', 'sum'),    
    mean_rew_eve=('reward', 'mean'),   
    median_rew_eve=('reward', 'median'), 
    # std_rew_eve=('reward', 'std'),         
    min_rew_eve=('reward', 'min'),         
    max_rew_eve=('reward', 'max'),         
    range_rew_eve=('reward', lambda x: x.max() - x.min()),  
    mean_t_eve=('time', 'mean'),
    median_t_eve=('time', 'median'),
    # std_t_eve=('time', 'std'),      
    min_t_eve=('time', 'min'),        
    max_t_eve=('time', 'max'),       
    range_t_eve=('time', lambda x: x.max() - x.min()),  
    freq_eve=('time', lambda x: (len(x) / (x.max() - x.min()))*100 if x.max() != x.min() else 0),
    last_to_end_eve=('time', lambda x: 714 - x.max())
    
).reset_index().round(2).fillna(0)

user_item_events.to_csv('medalion_data_store/silver/user_item_events.csv', index=False)

user_item_events

,person,ofr_id_short,cnt_eve,sum_rew_eve,mean_rew_eve,median_rew_eve,min_rew_eve,max_rew_eve,range_rew_eve,mean_t_eve,median_t_eve,min_t_eve,max_t_eve,range_t_eve,freq_eve,last_to_end_eve
0,0009655768c64bdeb2e877511632db8f,ofr_C,2,0.0,0.0,0.0,0.0,0.0,0.0,354.00,354.0,336,372,36,5.56,342
1,0009655768c64bdeb2e877511632db8f,ofr_G,3,2.0,2.0,2.0,2.0,2.0,0.0,524.00,528.0,504,540,36,8.33,174
2,0009655768c64bdeb2e877511632db8f,ofr_H,2,0.0,0.0,0.0,0.0,0.0,0.0,180.00,180.0,168,192,24,8.33,522
3,0009655768c64bdeb2e877511632db8f,ofr_I,3,5.0,5.0,5.0,5.0,5.0,0.0,426.00,414.0,408,456,48,6.25,258
4,0009655768c64bdeb2e877511632db8f,ofr_J,2,2.0,2.0,2.0,2.0,2.0,0.0,576.00,576.0,576,576,0,0.00,138
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63283,fffad4f4828548d1b5583907f2e9906b,ofr_I,6,10.0,5.0,5.0,5.0,5.0,0.0,246.00,222.0,0,516,516,1.16,198
63284,ffff82501cea40309d5fdd7edcca4a07,ofr_D,3,5.0,5.0,5.0,5.0,5.0,0.0,514.00,504.0,504,534,30,10.00,180
63285,ffff82501cea40309d5fdd7edcca4a07,ofr_E,3,5.0,5.0,5.0,5.0,5.0,0.0,180.00,174.0,168,198,30,10.00,516
63286,ffff82501cea40309d5fdd7edcca4a07,ofr_G,3,2.0,2.0,2.0,2.0,2.0,0.0,22.00,6.0,0,60,60,5.00,654


In [80]:
user_item_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63288 entries, 0 to 63287
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   person           63288 non-null  object 
 1   ofr_id_short     63288 non-null  object 
 2   cnt_eve          63288 non-null  int64  
 3   sum_rew_eve      63288 non-null  float64
 4   mean_rew_eve     63288 non-null  float64
 5   median_rew_eve   63288 non-null  float64
 6   min_rew_eve      63288 non-null  float64
 7   max_rew_eve      63288 non-null  float64
 8   range_rew_eve    63288 non-null  float64
 9   mean_t_eve       63288 non-null  float64
 10  median_t_eve     63288 non-null  float64
 11  min_t_eve        63288 non-null  int64  
 12  max_t_eve        63288 non-null  int64  
 13  range_t_eve      63288 non-null  int64  
 14  freq_eve         63288 non-null  float64
 15  last_to_end_eve  63288 non-null  int64  
dtypes: float64(9), int64(5), object(2)
memory usage: 7.7+ MB


## User-Item-Transactions matrix:

### A table that tracks transactions based on **person** columns. The aggregation process computes several statistics related to the **transactions amount**, and **time** for each user.
---

**Source table**: 

  `transactions` table: providing amount-spent and time-stamp per person transactions.

**Created table:**
    `user_item_transactions`: grouped table by `'person'` with counts and statistics features.

**Columns Created:**

1. **cnt_tran**: Total number of transactions per person.
2. **sum_am_tran**: Sum of transaction amounts.
3. **mean_am_tran**: Mean of transaction amounts.
4. **median_am_tran**: Median of transaction amounts.
5. **std_am_tran**: Standard deviation of transaction amounts. **(*removed*)**
6. **min_am_tran**: Minimum transaction amount.
7. **max_am_tran**: Maximum transaction amount.
8. **range_amount_tran**: Range of transaction amounts (max - min).
9. **mean_t_tran**: Mean transaction time.
10. **median_t_tran**: Median transaction time.
11. **std_t_tran**: Standard deviation of transaction times. **(*removed*)**
12. **min_t_tran**: Minimum transaction time.
13. **max_t_tran**: Maximum transaction time.
14. **range_t_tran**: Range of transaction times (max - min).
15. **freq_tran**: Frequency of transactions per person. If max time equals min time, frequency is set to 0 to avoid division by zero.
16. **last_to_end_tran**: last transaction occurrence before the end of offer program (end=714h).

**Handling Missing Values:**
  - fill NaN with zero.

In [81]:
# Group transaction data by person and event, aggregating count and sum of amounts
user_item_transactions = transactions.groupby(['person']).agg(
    cnt_tran=('event', 'count'),        
    sum_am_tran=('amount', 'sum'),    
    mean_am_tran=('amount', 'mean'),   
    median_am_tran=('amount', 'median'), 
    # std_am_tran=('amount', 'std'),         
    min_am_tran=('amount', 'min'),         
    max_am_tran=('amount', 'max'),         
    range_amount_tran=('amount', lambda x: x.max() - x.min()),  
    mean_t_tran=('time', 'mean'),
    median_t_tran=('time', 'median'),
    # std_t_tran=('time', 'std'),      
    min_t_tran=('time', 'min'),        
    max_t_tran=('time', 'max'),       
    range_t_tran=('time', lambda x: x.max() - x.min()),  
    freq_tran=('time', lambda x: (len(x) / (x.max() - x.min()))*100 if x.max() != x.min() else 0),
    last_to_end_tran=('time', lambda x: 714 - x.max())

).round(2).reset_index().fillna(0)

# save the table in the data store
user_item_transactions.to_csv('medalion_data_store/silver/user_item_transactions.csv', index=False)

# Output final user-item-transactions matrix
user_item_transactions

,person,cnt_tran,sum_am_tran,mean_am_tran,median_am_tran,min_am_tran,max_am_tran,range_amount_tran,mean_t_tran,median_t_tran,min_t_tran,max_t_tran,range_t_tran,freq_tran,last_to_end_tran
0,0009655768c64bdeb2e877511632db8f,8,127.60,15.95,13.84,8.57,28.16,19.59,543.00,564.0,228,696,468,1.71,18
1,00116118485d4dfda04fdbaba9a87b5c,3,4.09,1.36,0.70,0.20,3.19,2.99,408.00,456.0,294,474,180,1.67,240
2,0011e0d4e6b944f998e987f904e8c1e5,5,79.46,15.89,13.49,8.96,23.03,14.07,451.20,576.0,132,654,522,0.96,60
3,0020c2b971eb4e9188eac86d93036a77,8,196.86,24.61,24.35,17.24,33.86,16.62,348.75,327.0,54,708,654,1.22,6
4,0020ccbbb6d84e358d3414a3ff76cffd,12,154.05,12.84,12.76,6.81,20.08,13.27,375.00,402.0,42,672,630,1.90,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16573,fff3ba4757bd42088c044ca26d73817a,11,580.98,52.82,20.98,10.99,388.22,377.23,246.00,192.0,6,552,546,2.01,162
16574,fff7576017104bcc8677a8d63322b5e1,6,29.94,4.99,5.03,2.08,8.01,5.93,392.00,417.0,36,696,660,0.91,18
16575,fff8957ea8b240a6b5e634b6ee8eafcf,5,12.15,2.43,0.89,0.64,6.39,5.75,379.20,450.0,18,576,558,0.90,138
16576,fffad4f4828548d1b5583907f2e9906b,12,88.83,7.40,7.52,2.05,12.18,10.13,323.50,318.0,36,678,642,1.87,36


## User-Item-U-Event matrix:

A table that tracks **unique** interactions by person-offer-event over time to create features that analyse user engagement and responsiveness for each unique offer interaction, person by person.

---

* **Source table**: 

    `events` table: event type and time per person interactions with offers.

* **Created table:**
    
   `user_item_event`: table of *facts* of unique person-offer-event interactions.

### Fact Definition:

-  A **Fact** represents a unique ocurrence sequence when a person engages with an single offer, and is tracked from the moment the offer is received, viewed, and completed. The interactions that occur more than once is tracked as a unique fact.
- The **'tag'** column created ensures the uniqueness of the fact and *prevents aggregation when a person interacts with the same offer type more than once*. As the aggregation using the tag column, it returns a single value, using agg with 'max', 'min' or 'mean' will return the same value for a fact.

### Engagement Metrics Feature Engineering Strategy: Key features to quantify user responsiveness.

- Each *person-event-offer-tag* fact is grouped and stacked into `event` columns, placing `time` and `reward` values accordingly.
- Time-event colunm are created and missing values computed as `NaN` value.


  - **Handling Missing Values:**
    - Before features calculations, the stacked columns `time_completed`, `time_received`, and `time_viewed` columns have missing values filled with `np.inf` (never ended event), avoiding *artificial* values imputies.

  - **Time-Differences-help-features** - not included in the final dataset as it contains np.inf values
    - `to_vr`: Time from receiving to viewing - delays in viewing - {0 to inf}
    - `to_cv`: Time from viewing to completion - delays in completions after viewing - {-inf to inf}
    - `to_cr`: Time from receiving to completion - delays in completion, even if viewed or not - {0 to inf}

  - **Inverse Time-Based Scores (`1/(x+1) * 100`):** calculated using the previous time differences where x is the time difference.
    - `curiosity_vr`: `{0 to 100}` Measures _speed_ of viewing after receiving.
    - `eagerness_cv`: `{0 to 100} or -1` Measures _speed_ of completion after viewing (`-1` indicates completion before viewing).
    - `overall_cr`: `{0 to 100}` Overall responsiveness from receiving to completion.

  - **Influence Metrics:**
    - **`influence`**: Binary flag (1/0) indicating if an offer was completed after viewing (responsiveness).
    - **`ext_influence`**: Binary flag capturing simultaneous viewing and completion time (extreme responsiveness).

  - **Counts:** (by fact using tag column to avoid aggregation on the same offer type per person interactions).
    - `count_offer_completed`: Count of completed offers.
    - `count_offer_received`: Count of received offers. 
    - `count_offer_viewed`: Count of viewed offers.

---

> NOTE: This table have features that can be combined to determine which demographic groups respond best to which offer type.

In [82]:
# Grouping events by 'person', 'ofr_id_short', 'event', and 'tag', then aggregating the maximum 'time' and 'reward' per group and restructuring the DataFrame.
user_item_u_event = (events.groupby(['person', 'event', 'ofr_id_short', 'tag'], dropna=False)
    .agg(
        cnt = ('event', 'count'),
        time=('time', lambda x: x),
    )
    .unstack(level=1)  # Unstacking by 'event' to widen the DataFrame
    .reset_index()
)

# Flattening the multi-level column names
user_item_u_event.columns = ['_'.join(col).strip('_') for col in user_item_u_event.columns.to_flat_index()]

# computing the stage of the fact: 1- just received, 2- received-viwed or received-completed, 3- all occurrences
user_item_u_event['stage'] = user_item_u_event[['cnt_completed',	'cnt_received',	'cnt_viewed']].sum(axis=1)

# Replacing NaN values with infinity to facilitate time calculations
user_item_u_event['time_completed'] = user_item_u_event['time_completed'].fillna(np.inf)
user_item_u_event['time_received'] = user_item_u_event['time_received'].fillna(np.inf)
user_item_u_event['time_viewed'] = user_item_u_event['time_viewed'].fillna(np.inf)

# Computing time differences for various stages of the offer lifecycle
user_item_u_event['to_vr'] = user_item_u_event['time_viewed'] - user_item_u_event['time_received']
user_item_u_event['to_cv'] = user_item_u_event['time_completed'] - user_item_u_event['time_viewed']
user_item_u_event['to_cr'] = user_item_u_event['time_completed'] - user_item_u_event['time_received']

# Calculating curiosity, eagerness, and overall responsiveness scores
user_item_u_event['curiosity_vr'] = (1 / (user_item_u_event['to_vr'] + 1)) * 100
user_item_u_event['eagerness_cv'] = [(1 / (x + 1))*100 if ((not np.isnan(x)) & (x >=0)) else -1 for x in user_item_u_event['to_cv']]
user_item_u_event['overall_cr'] = (1 / (user_item_u_event['to_cr'] + 1)) * 100


# Defining influence metrics based on time conditions
user_item_u_event['influence'] = (
    (~pd.isna(user_item_u_event['time_completed'])) & 
    (user_item_u_event['time_completed'] != np.inf) &
    (user_item_u_event['time_completed'] >= user_item_u_event['time_viewed'])
).astype(int)

# Extreme influence is when the offer is viewed and completed at the same time.
user_item_u_event['ext_influence'] = (
    (~pd.isna(user_item_u_event['time_viewed'])) & 
    (user_item_u_event['time_viewed'] != np.inf) & 
    (user_item_u_event['time_completed'] == user_item_u_event['time_viewed'])
).astype(int)

# Dropping intermediate time calculation columns and others, rounding final values.
user_item_u_event = user_item_u_event.drop(
    columns=['time_completed', 'time_received', 'time_viewed', 'to_vr', 'to_cv', 'to_cr']
).round(1)

user_item_u_event = user_item_u_event.round(2).fillna(0)

# Persisting the data in the gold layer
user_item_u_event.to_csv('medalion_data_store/silver/user_item_u_event.csv', index=False)



# Displaying the final DataFrame
user_item_u_event

,person,ofr_id_short,tag,cnt_completed,cnt_received,cnt_viewed,stage,curiosity_vr,eagerness_cv,overall_cr,influence,ext_influence
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,0.0,1.0,1.0,2.0,2.7,0.0,0.0,0,0
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,1.0,1.0,1.0,3.0,2.7,-1.0,4.0,0,0
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,0.0,1.0,1.0,2.0,4.0,0.0,0.0,0,0
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,1.0,1.0,1.0,3.0,2.0,-1.0,14.3,0,0
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,1.0,1.0,0.0,2.0,0.0,-1.0,100.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,1.0,1.0,1.0,3.0,14.3,4.0,3.2,1,0
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,1.0,1.0,1.0,3.0,14.3,1.8,1.6,1,0
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,1.0,1.0,1.0,3.0,5.3,3.2,2.0,1,0
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,1.0,1.0,1.0,3.0,14.3,100.0,14.3,1,1


## Transactions-Time matrix

Constructing a **Transaction-Time** based matrix from `transaction` table. The time frame was divided in 20 periods. The first period conteins all times. The second has 36 hours less than the first time, and so on. The sum of transactions per period was computed. 

---
* **Source**: 

    `transaction` table.

* **Created tables:**
    
    `transactions_time` with related features.

    
### Feature Engineering strategy:
**periods:** The values in each period changes tracked by line.
**churn:** if no transactions in the three last periods was found compute 1 else 0. 
**recency:** The time to end of the offer program from the last transaction (714-time). 
**churn2:** if recency is more then 96h compute 1 else 0.

### Data Processing
- The `.unstack()` function pivots the `event` or the `ofr_id_short` column, converting different event/offer types into separated columns and filling it's values as above.

- No further calculations are performed on the data columns, as the summary statistics are computed directly from the groupby/unstack operations.

- The resulting multi-level column names are flattened using list comprehension to create more readable column names.


---

In [94]:
# Aggregate transaction counts per user over time
transactions_time = (
    transactions.groupby(['person', 'time'])
    .size()
    .unstack(level=1)
    #.reset_index()
)

# Rename columns to indicate transaction counts over time
transactions_time.columns = [
    f"time_tran_{col}" for col in transactions_time.columns.to_flat_index()
]

# Compute transaction frequencies over defined periods (~60 hours per period)
frequences_period = pd.DataFrame({
    f'period_{i//6 + 1}': transactions_time.iloc[:, i+1:].sum(axis=1)
    for i in range(0, 119, 6)
}).reset_index()

# Define churn as 1 if no transactions occurred in the last three periods
churn = pd.Series((frequences_period.iloc[:, -3:].sum(axis=1) == 0).astype(int), name='churn')

# Compute recency: Time elapsed since last transaction (714 - max transaction time per person)
recency = (714 - transactions.groupby('person')['time'].max()).reset_index(drop=True)
recency.name = 'recency'

# Alternative churn definition: 1 if recency is greater than 96 hours
churn2 = pd.Series((recency > 96).astype(int), name='churn2')

# Combine all features into a final churn prediction table
transactions_time = pd.concat([
        frequences_period, recency, churn, churn2
], axis=1)

# Save churn data to a CSV file
transactions_time.to_csv('medalion_data_store/silver/transactions_time.csv', index=False)

# Output the churn table
transactions_time


,person,period_1,period_2,period_3,period_4,period_5,period_6,period_7,period_8,period_9,...,period_14,period_15,period_16,period_17,period_18,period_19,period_20,recency,churn,churn2
0,0009655768c64bdeb2e877511632db8f,8.0,8.0,8.0,8.0,8.0,8.0,8.0,7.0,7.0,...,6.0,6.0,5.0,3.0,3.0,3.0,2.0,18,0,0
1,00116118485d4dfda04fdbaba9a87b5c,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,240,1,1
2,0011e0d4e6b944f998e987f904e8c1e5,5.0,5.0,5.0,5.0,4.0,4.0,4.0,3.0,3.0,...,3.0,3.0,3.0,2.0,2.0,1.0,0.0,60,0,0
3,0020c2b971eb4e9188eac86d93036a77,8.0,8.0,5.0,5.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,2.0,2.0,2.0,2.0,2.0,6,0,0
4,0020ccbbb6d84e358d3414a3ff76cffd,12.0,12.0,11.0,11.0,11.0,11.0,11.0,9.0,8.0,...,2.0,2.0,2.0,2.0,1.0,1.0,0.0,42,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16573,fff3ba4757bd42088c044ca26d73817a,11.0,9.0,8.0,7.0,7.0,6.0,5.0,4.0,4.0,...,3.0,3.0,1.0,0.0,0.0,0.0,0.0,162,1,1
16574,fff7576017104bcc8677a8d63322b5e1,6.0,5.0,5.0,5.0,5.0,5.0,4.0,4.0,3.0,...,3.0,3.0,3.0,2.0,1.0,1.0,1.0,18,0,0
16575,fff8957ea8b240a6b5e634b6ee8eafcf,5.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,...,2.0,1.0,1.0,0.0,0.0,0.0,0.0,138,1,1
16576,fffad4f4828548d1b5583907f2e9906b,12.0,11.0,10.0,9.0,8.0,8.0,7.0,7.0,7.0,...,4.0,4.0,3.0,2.0,1.0,1.0,0.0,36,0,0


# Analytical table 

A final table mergin selected tables created in the previous steps.

The goals is get different types of dataset to be used in the analysis, modeling and recommendations.

In [97]:
# Merge unique event features with transcript features based on 'person' column
# Use a left join to keep all records from user_item_event
analytical_table = user_item_u_event.merge(profile, left_on='person', right_on='id', how='left').drop(columns=['id'])

analytical_table = analytical_table.merge(user_item_events, on=['person', 'ofr_id_short'], how='left')

analytical_table = analytical_table.merge(user_item_transactions, on=['person'], how='left')

analytical_table = analytical_table.merge(transactions_time, on=['person'], how='left')

# # Save the final dataset to a CSV file
analytical_table.to_csv('medalion_data_store/gold/analytical_table.csv', index=False)


analytical_table

,person,ofr_id_short,tag,cnt_completed,cnt_received,cnt_viewed,stage,curiosity_vr,eagerness_cv,overall_cr,...,period_14,period_15,period_16,period_17,period_18,period_19,period_20,recency,churn,churn2
0,0009655768c64bdeb2e877511632db8f,ofr_C,0,0.0,1.0,1.0,2.0,2.7,0.0,0.0,...,6.0,6.0,5.0,3.0,3.0,3.0,2.0,18.0,0.0,0.0
1,0009655768c64bdeb2e877511632db8f,ofr_G,0,1.0,1.0,1.0,3.0,2.7,-1.0,4.0,...,6.0,6.0,5.0,3.0,3.0,3.0,2.0,18.0,0.0,0.0
2,0009655768c64bdeb2e877511632db8f,ofr_H,0,0.0,1.0,1.0,2.0,4.0,0.0,0.0,...,6.0,6.0,5.0,3.0,3.0,3.0,2.0,18.0,0.0,0.0
3,0009655768c64bdeb2e877511632db8f,ofr_I,0,1.0,1.0,1.0,3.0,2.0,-1.0,14.3,...,6.0,6.0,5.0,3.0,3.0,3.0,2.0,18.0,0.0,0.0
4,0009655768c64bdeb2e877511632db8f,ofr_J,0,1.0,1.0,0.0,2.0,0.0,-1.0,100.0,...,6.0,6.0,5.0,3.0,3.0,3.0,2.0,18.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76272,ffff82501cea40309d5fdd7edcca4a07,ofr_E,0,1.0,1.0,1.0,3.0,14.3,4.0,3.2,...,5.0,3.0,3.0,2.0,1.0,0.0,0.0,66.0,0.0,0.0
76273,ffff82501cea40309d5fdd7edcca4a07,ofr_G,0,1.0,1.0,1.0,3.0,14.3,1.8,1.6,...,5.0,3.0,3.0,2.0,1.0,0.0,0.0,66.0,0.0,0.0
76274,ffff82501cea40309d5fdd7edcca4a07,ofr_J,0,1.0,1.0,1.0,3.0,5.3,3.2,2.0,...,5.0,3.0,3.0,2.0,1.0,0.0,0.0,66.0,0.0,0.0
76275,ffff82501cea40309d5fdd7edcca4a07,ofr_J,1,1.0,1.0,1.0,3.0,14.3,100.0,14.3,...,5.0,3.0,3.0,2.0,1.0,0.0,0.0,66.0,0.0,0.0
